<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#TO-DO-to-complete-NLP-prototype" data-toc-modified-id="TO-DO-to-complete-NLP-prototype-1">TO DO to complete NLP prototype</a></span></li><li><span><a href="#Explore-NLP-methods/vizs-with-champaign-restaurant-review-data" data-toc-modified-id="Explore-NLP-methods/vizs-with-champaign-restaurant-review-data-2">Explore NLP methods/vizs with champaign restaurant review data</a></span><ul class="toc-item"><li><span><a href="#Get-data" data-toc-modified-id="Get-data-2.1">Get data</a></span></li><li><span><a href="#Data-preview" data-toc-modified-id="Data-preview-2.2">Data preview</a></span></li><li><span><a href="#Tokenize" data-toc-modified-id="Tokenize-2.3">Tokenize</a></span></li><li><span><a href="#Make-dictionary-&amp;-corpus" data-toc-modified-id="Make-dictionary-&amp;-corpus-2.4">Make dictionary &amp; corpus</a></span></li><li><span><a href="#Get-Similarities" data-toc-modified-id="Get-Similarities-2.5">Get Similarities</a></span></li><li><span><a href="#Visualizations" data-toc-modified-id="Visualizations-2.6">Visualizations</a></span></li></ul></li></ul></div>

# TO DO to complete NLP prototype

- query similarities using combinations
- put edge data into networkx
- make plotly network viz
- add a few plotly node features
- clean up/refactor code

# Explore NLP methods/vizs with champaign restaurant review data

In [ ]:
#data 
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import pandas as pd

#spacy for tokenization
from spacy.lang.en import English # Create the nlp object
import spacy
#gensim for similarity
from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim.similarities.docsim import MatrixSimilarity,Similarity

#sklearn for distance metrics
from sklearn.metrics.pairwise import cosine_similarity

## Get data

In [ ]:
postgres_db_params = {'drivername': 'postgres',
                'database':'restaurants',
               'username': 'michaelkranz',
               'password': 'helloworld',
               'host': 'localhost',
               'port': 5432}

postgres_db_url = URL(**postgres_db_params)
engine = create_engine(postgres_db_url)

In [ ]:
review_df = pd.read_sql(
    con=engine.connect(),
    sql='''
    SELECT *
    FROM champaign_restaurant_reviews
    ''')

## Data preview

In [ ]:
review_df.head(2)

In [ ]:
review_df.head(2).text.values[0]

## Tokenize

In [ ]:
def tokenize_text(text_str,nlp_obj):
    '''
    use spacy to separate text into words
    (ie tokenization)
    and return the lemmatization 
    (ie feet for footing and foot)
    for only nouns and adjectives
    
    TODO: refine methodology
    '''
    spacy_doc = nlp_obj(text_str)
    
    tokenized_doc = [
        token.lemma_
        for token in spacy_doc
        if token.pos_ in ("NOUN","ADJ")
        ]
    
    return tokenized_doc
    #return spacy_doc
        

## Make dictionary & corpus

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
#TODO: combine reviews in SQL to scale
#TODO: sciktilearn?

In [ ]:
reviews_tokenized = (
    review_df
    #.head(2)
    .groupby('business_id')
    .text
    .apply(lambda x: ' '.join(x))
    .apply(tokenize_text,nlp_obj=nlp)
)

In [ ]:
review_df.query("business_id=='-5NXoZeGBdx3Bdk70tuyCw'")

In [ ]:
review_df.query("business_id=='-2q4dnUw0gGJniGW2aPamQ'")

In [ ]:
review_df.query("business_id=='-2q4dnUw0gGJniGW2aPamQ'")

In [ ]:
reviews_dictionary = Dictionary(reviews_tokenized)

In [ ]:
reviews_dictionary.num_docs

In [ ]:
review_df.business_id.unique().shape

In [ ]:
#corpus
reviews_corpus = [reviews_dictionary.doc2bow(doc) for doc in reviews_tokenized]

In [ ]:
#tfidf with document being each restaurant and corpus being all restaurants
reviews_tfidf_model = TfidfModel(reviews_corpus)

In [ ]:
reviews_tfidf_docs = [reviews_tfidf_model[review] for review in reviews_corpus]

## Get Similarities

In [ ]:
#similarity indices for each doc
similarity_indices = MatrixSimilarity(reviews_tfidf_docs)

In [ ]:
#doc example
fiesta = reviews_corpus[doc_id]
fiesta_tfidf = reviews_tfidf_model[fiesta]
similarity_indices[fiesta_tfidf]

In [ ]:
#similarity to each restaurant (ie doc)
fiesta_similarity = (
    pd.DataFrameame(similarity_indices[fiesta_tfidf])
    .assign(index=doc_id)
)

In [ ]:
fiesta_similarity

In [ ]:
fiesta_similarity.sort_values(ascending=False).index

In [ ]:
doc_mapping.iloc[[203, 587, 647, 111]]

In [ ]:
#get index:name mappings
doc_mapping = (
    review_df
    .groupby('business_id')
    ['name']
    .apply(lambda x: x.unique()[0])
)

#ie switch index to key and token str to object
token_mapping = {
    i:token 
    for token,i in reviews_dictionary.token2id.items()
} 

In [ ]:
#for orange and brew and bevande coffee (doc indices 0 and 172):
#doc_id = 0
all_bag_of_words_list = []
for doc_id in range(len(reviews_corpus)):
    bag_of_words = (
        pd.DataFrame(
            {"frequency":dict(reviews_corpus[doc_id]),
             "tf_idf":dict(reviews_tfidf_docs[doc_id]),
             "business":doc_mapping[doc_id],
             "word":token_mapping
            }
        )
        .set_index(['business','word'])
    )
    all_bag_of_words_list.append(bag_of_words)                                 

In [ ]:
all_bag_of_words_df = pd.concat(all_bag_of_words_list)

In [ ]:
fiesta_similarity

In [ ]:
# Function which returns subset or r length from n 
from itertools import combinations

In [2]:
test = [1,2,3,4]

In [4]:
list(combinations(test,r=2))

[(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]

In [ ]:
all_similarities_list = []
for doc_id in range(len(reviews_corpus)):
    #doc example
    doc = reviews_corpus[doc_id]
    doc_tfidf = reviews_tfidf_model[doc]
    similarities = similarity_indices[doc_tfidf]
    
    all_similarities_list.append(similarities)    

## Visualizations

In [ ]:
import plotly.express as px

In [ ]:
restaurants = ['Orange & Brew','Bevande Coffee']

In [ ]:
plot_df = (
    all_bag_of_words_df
    .loc[restaurants]
    .unstack('business')
    .reset_index()
)

plot_df.columns = [
    '_'.join([x for x in column if len(x)>0]) 
    for column in plot_df.columns
]

In [ ]:
plot_df.head(2)

In [ ]:
hover_data_list = [
    'word',
    'frequency_Orange & Brew',
    'frequency_Bevande Coffee'
]

#x_str = "frequency_Orange & Brew"
#y_str = "frequency_Bevande Coffee"

x_str = "tf_idf_Orange & Brew"
y_str = "tf_idf_Bevande Coffee"

In [ ]:
plot_df.columns

In [ ]:
plot_df.word.unique().shape

In [ ]:
fig = px.scatter(plot_df,
                 x=x_str, 
                 y=y_str, 
                 #color="species",
                 #size='petal_length', 
                 hover_data=hover_data_list)
fig.show()

In [ ]:
#ratings

#location